In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# 데이터 불러오기
df = pd.read_csv('/content/drugLibTrain_raw_new.tsv', sep="\t", encoding='latin1')

# 필요한 열만 선택
data = df[['urlDrugName', 'benefitsReview', 'sideEffectsReview', 'commentsReview', 'condition']]

# 불필요한 데이터 제거 (na 값 등)
data = data.dropna()

# 데이터 전처리
# 각 리뷰를 하나의 텍스트로 합침
data['all_reviews'] = data['benefitsReview'] + ' ' + data['sideEffectsReview'] + ' ' + data['commentsReview']

# Tokenizer 객체 생성
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['all_reviews'])

# 텍스트를 시퀀스로 변환
sequences = tokenizer.texts_to_sequences(data['all_reviews'])

# 시퀀스의 최대 길이 계산
max_length = max(len(seq) for seq in sequences)

# 시퀀스를 일정한 길이로 패딩
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# 라벨 인코딩
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['urlDrugName'])

# 조건 데이터를 범주형 변수로 변환
conditions = pd.get_dummies(data['condition'])

# 리뷰 데이터와 조건 데이터를 결합
X = np.hstack((padded_sequences, conditions.values))

# 데이터를 훈련 및 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# 모델 생성
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])


Epoch 1/50
38/38 [==============================] - 1s 13ms/step - loss: 228.8053 - accuracy: 0.0050 - val_loss: 25.2577 - val_accuracy: 0.0166
Epoch 2/50
38/38 [==============================] - 0s 8ms/step - loss: 16.8327 - accuracy: 0.0187 - val_loss: 10.9782 - val_accuracy: 0.0216
Epoch 3/50
38/38 [==============================] - 0s 8ms/step - loss: 8.9461 - accuracy: 0.0225 - val_loss: 9.0507 - val_accuracy: 0.0199
Epoch 4/50
38/38 [==============================] - 0s 8ms/step - loss: 7.5576 - accuracy: 0.0229 - val_loss: 8.3177 - val_accuracy: 0.0216
Epoch 5/50
38/38 [==============================] - 0s 8ms/step - loss: 6.6933 - accuracy: 0.0233 - val_loss: 7.9626 - val_accuracy: 0.0199
Epoch 6/50
38/38 [==============================] - 0s 7ms/step - loss: 6.3725 - accuracy: 0.0266 - val_loss: 7.7767 - val_accuracy: 0.0233
Epoch 7/50
38/38 [==============================] - 0s 7ms/step - loss: 6.2667 - accuracy: 0.0266 - val_loss: 7.6796 - val_accuracy: 0.0199
Epoch 8/50
38/